In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

2024-03-12 11:13:59.326855: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 11:13:59.568315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 11:13:59.569336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 11:14:00.796535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:

# Define paths to your parallel dataset corpus
train_en_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/train.am-en.base.en"
train_am_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/train.am-en.base.am"
test_en_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/test.am-en.base.en"
test_am_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/test.am-en.base.am"
dev_en_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/dev.am-en.base.en"
dev_am_path = "data/Amharic–English_Parallel_Corpus-version_1.0 (1)/Amharic–English_Parallel_Corpus-version_1.0/dev.am-en.base.am"

In [7]:

# Hyperparameters (adjust as needed)
MAX_LEN = 200  # Maximum sentence length
EMBEDDING_DIM = 128  # Embedding dimension for words
LSTM_UNITS = 256  # Number of units in the LSTM layer
BATCH_SIZE = 64  # Batch size for training
EPOCHS = 10  # Number of training epochs


In [9]:

# Load English and Amharic text data
def load_data(en_path, am_path):
    with open(en_path, "r", encoding="utf-8") as f:
        en_lines = f.readlines()
    with open(am_path, "r", encoding="utf-8") as f:
        am_lines = f.readlines()
    return en_lines, am_lines

en_train, am_train = load_data(train_en_path, train_am_path)
en_test, am_test = load_data(test_en_path, test_am_path)
en_dev, am_dev = load_data(dev_en_path, dev_am_path)

In [11]:

# Preprocess text data (clean, tokenize, pad)
def preprocess_data(text_data):
    tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(text_data)
    sequences = tokenizer.texts_to_sequences(text_data)
    padded = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')
    return padded, tokenizer

en_train_padded, en_tokenizer = preprocess_data(en_train)
am_train_padded, am_tokenizer = preprocess_data(am_train)
en_test_padded, _ = preprocess_data(en_test)  # Reuse existing tokenizer
am_test_padded, _ = preprocess_data(am_test)  # Reuse existing tokenizer
en_dev_padded, _ = preprocess_data(en_dev)  # Reuse existing tokenizer
am_dev_padded, _ = preprocess_data(am_dev)  # Reuse existing tokenizer

# Train the model for


In [13]:

# Create English-Amharic (forward) and Amharic-English (reverse) vocabulary sizes
en_vocab_size = len(en_tokenizer.word_index) + 1
am_vocab_size = len(am_tokenizer.word_index) + 1

# Define the machine translation model
def create_model(src_vocab_size, target_vocab_size):
    model = Sequential()
    model.add(Embedding(src_vocab_size, EMBEDDING_DIM, input_length=MAX_LEN))
    model.add(LSTM(LSTM_UNITS, return_sequences=True))
    model.add(LSTM(LSTM_UNITS))
    model.add(Dense(target_vocab_size, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [14]:

# Train the model for English-Amharic translation
en_to_am_model = create_model(en_vocab_size, am_vocab_size)

2024-03-12 11:18:54.570032: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [15]:
en_to_am_model.fit(en_train_padded, am_train_padded, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(en_dev_padded, am_dev_padded))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/runpy.py", line 193, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/torna

In [16]:
import tensorflow as tf
from transformers import MarianMTModel, MarianTokenizer

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Moses installation (may require additional steps)
!pip install mosesdecoder

ERROR: Could not find a version that satisfies the requirement mosesdecoder (from versions: none)
ERROR: No matching distribution found for mosesdecoder


In [18]:
# Assuming your data is in a text format with each line representing a sentence pair (English, Amharic)
with open(train_en_path, "r") as f_en, open(train_am_path, "r") as f_am:
    english_sentences = f_en.readlines()
    amharic_sentences = f_am.readlines()


In [20]:
import nltk  # For tokenization
from collections import Counter

def clean_text(text):
    # Tokenize
    tokens = nltk.word_tokenize(text.lower())
    # Remove punctuation (optional)
    # tokens = [t for t in tokens if t.isalpha()]
    return tokens

In [28]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/babi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/babi/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/babi/nltk_data...


True

In [32]:
len(amharic_sentences)

140000

In [30]:
english_cleaned = [clean_text(s) for s in english_sentences]
amharic_cleaned = [clean_text(s) for s in amharic_sentences]


# Build vocabulary (example for NMT)
word_counts = Counter()
for sentence in english_cleaned + amharic_cleaned:
    word_counts.update(sentence)
vocabulary = [w for w, c in word_counts.items() if c >= 5]  # Adjust frequency threshold as needed


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(english_cleaned, amharic_cleaned, test_size=0.1, random_state=42)

In [44]:
model_name = "Helsinki-NLP/opus-mt-am-sv"  # Pre-trained model for English-Amharic translation
tokenizer = MarianTokenizer.from_pretrained(model_name)

def prepare_dataset(sentences, tgt_lang="am"):  # Adjust tgt_lang as needed
    encoded_data = tokenizer(sentences, padding="longest", return_tensors="tf")
    return encoded_data

train_data = prepare_dataset(X_train)
val_data = prepare_dataset(X_val)

model = MarianMTModel.from_pretrained(model_name)

# Define optimizer and loss function (example)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Training loop (example)
EPOCHS = 5  # Adjust as needed
for epoch in range(EPOCHS):
    with tf.GradientTape() as tape:
        outputs = model(train_data["input_ids"], labels=train_data["labels"])
        loss = loss_fn(train_data["labels"], outputs.logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # Evaluation on validation set (optional)

# Save the trained model (optional)
model.save_pretrained("my_amharic_english_model")


ValueError: too many values to unpack (expected 2)

In [ ]:
def translate(sentence, model, tokenizer):
    encoded_input = tokenizer(sentence,)